In [1]:
%%bash

pip install deeppavlov
pip install tensorflow-hub

  Running setup.py bdist_wheel for pytelegrambotapi: started
  Running setup.py bdist_wheel for pytelegrambotapi: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/b0/a0/fa/c3539fd47aa9f834230d64039c4bc620463bc7afc39b0f3f35
  Running setup.py bdist_wheel for overrides: started
  Running setup.py bdist_wheel for overrides: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/8d/52/86/e5a83b1797e7d263b458d2334edd2704c78508b3eea9323718
  Running setup.py bdist_wheel for nltk: started
  Running setup.py bdist_wheel for nltk: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/da/8c/38/a8a36581975f8d03275c49960019f955e4d19fd14ae7e42d3d
  Running setup.py bdist_wheel for pycparser: started
  Running setup.py bdist_wheel for pycparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pytelegrambotapi overrides nltk pycpa

tensorflow-gpu 1.12.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
tensorflow-gpu 1.12.0 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.1 which is incompatible.
imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
You are using pip version 18.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Initialization

In [3]:
%load_ext autoreload
%autoreload 2

# Use only one GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../isanlp/src/')
sys.path.append('../../src/isanlp_srl_framebank/')
sys.path.append('../../libs/')
sys.path.append('../../libs/pylingtools/')

In [4]:
# Supress tensorflow memory appetites

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.log_device_placement=True
sess = tf.Session(config=config)

from tensorflow.python.keras import backend as K
K.set_session(sess)

In [75]:
tf.__version__

'1.12.0'

In [5]:
# Check available GPUs

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [11]:
import os
import time
import isanlp
import json
import pickle

import numpy as np
np.random.seed(31)

import pandas as pd

In [12]:
from tqdm import tqdm_notebook as tqdm

# Data loading

In [13]:
cleared_corpus_path = '../../data/cleared_corpus.json'

In [14]:
with open(cleared_corpus_path, 'r') as f:
    examples = json.load(f)

In [15]:
ling_data_path = '../../data/results_final_fixed.pckl'
with open(ling_data_path, 'rb') as f:
    ling_data_cache = pickle.load(f)

ling_data_cache = {k: v for k,v in ling_data_cache}

In [16]:
error_examples = {}

# Feature construction

In [17]:
from isanlp.annotation_repr import CSentence
from convert_corpus_to_brat import make_text


def find_address_by_offset(offset, ling_ann):
    for tok_num, tok in enumerate(ling_ann['tokens']):
        if tok.begin <= offset and offset < tok.end:
            break
    
    for sent_num, sent in enumerate(ling_ann['sentences']):
        if sent.begin <= tok_num and tok_num < sent.end:
            break
    
    return sent_num, tok_num - sent.begin


def process_arg_pred(feature_extractor, ling_cache, ex_id, pred, args, example):
    feature_sets = list()
    
    text, offset_index = make_text(example, 0)
    ling_ann = ling_cache[ex_id]
    
    pred_offset = offset_index[(pred[0], pred[1])]
    pred_ling_sent, pred_ling_word = find_address_by_offset(pred_offset, ling_ann)
    
    for arg in args:
        arg_offset = offset_index[(arg[0], arg[1])]
        arg_ling_sent, arg_ling_word = find_address_by_offset(arg_offset, ling_ann)
        
        fb_pred_word = example[pred[0]][pred[1]]
        fb_arg_word = example[arg[0]][arg[1]]
        
        if arg_ling_sent != pred_ling_sent:
            error_examples[ex_id] = {
                'reason': 'sent_mismatch',
                'arg': arg_ling_sent,
                'pred': pred_ling_sent
            }
            continue
            
        sentence = ling_ann['sentences'][pred_ling_sent]
        tokens = [tok.text for tok in ling_ann['tokens']]
        tokens = tokens[sentence.begin:sentence.end]
        
        role = fb_arg_word['rolepred1']

        features = feature_extractor.extract_features(pred_ling_word, 
                                                      arg_ling_word, 
                                                      ling_ann['postag'][arg_ling_sent],
                                                      ling_ann['morph'][arg_ling_sent],
                                                      ling_ann['lemma'][arg_ling_sent],
                                                      ling_ann['syntax_dep_tree'][arg_ling_sent])

                    
        feature_sets.append((features, role, ex_id, tokens, arg_ling_word, pred_ling_word))
    
    return feature_sets


def process_example(feature_extractor, ling_cache, ex_id, sentences):
    pred = None
    args = list()
    for sent_num, sent in enumerate(sentences):
        for word_num, word in enumerate(sent):
            if 'rank' in word and word['rank'] == 'Предикат':
                pred = (sent_num, word_num)
            elif 'rolepred1' in word:
                args.append((sent_num, word_num))
    
    return process_arg_pred(feature_extractor, ling_cache, ex_id, pred, args, sentences)


num_of_errors = 0
def prepare_train_data(examples, ling_data_cache, feature_extractor):
    feature_sets = []
    for ex_num, (ex_id, ex) in tqdm(list(enumerate(examples))):                
        feature_sets += process_example(feature_extractor, ling_data_cache, ex_id, ex)

    print('Number of training examples:', len(feature_sets))
    return feature_sets

In [18]:
main_model_path_root = '../../data/models_new/'

In [19]:
%autoreload

In [64]:
#!!!: Choose feature model here
from isanlp_srl_framebank.processor_srl_framebank import FeatureModelDefault
feature_model = FeatureModelDefault()
main_model_path = os.path.join(main_model_path_root, 'known_preds')

# from isanlp_srl_framebank.processor_srl_framebank import FeatureModelUnknownPredicates
# feature_model = FeatureModelUnknownPredicates()
# main_model_path = os.path.join(main_model_path_root, 'unknown_preds')

with open(os.path.join(main_model_path, 'feature_model.pckl'), 'wb') as f:
    pickle.dump(feature_model, f)

In [21]:
(vars(ling_data_cache['30587']['sentences'][0]))

{'begin': 0, 'end': 17}

In [22]:
len(error_examples)

0

In [23]:
feature_sets = prepare_train_data(examples, ling_data_cache, feature_model)

data_for_pandas = []
for example in feature_sets:
    data_for_pandas_ex = {}
    data_for_pandas_ex['role'] = example[1]
    data_for_pandas_ex['ex_id'] = example[2]
    data_for_pandas_ex['tokens'] = example[3]
    data_for_pandas_ex['arg_address'] = example[4]
    data_for_pandas_ex['prd_address'] = example[5]
    for elem in example[0]:
        for subelem in elem:
            if subelem is not None:
                data_for_pandas_ex.update(subelem)
    
    data_for_pandas.append(data_for_pandas_ex)
    
pd_data = pd.DataFrame(data_for_pandas)
del data_for_pandas
pd_data = pd_data.sample(frac=1)
pd_data[:3]


Number of training examples: 57552


,Animacy_arg,Aspect_arg,Gender_arg,Number_arg,Tense_arg,Valency_arg,VerbForm_arg,arg_address,arg_case,arg_lemma,...,dist,ex_id,prd_address,pred_lemma,pred_pos,prepos,rel_pos,role,syn_link_name,tokens
13454,Inan,,Neut,Sing,,,,30,Acc,лицо_NOUN,...,3.0,137553,27,продемонстрировать_VERB,VERB,,-1.0,атрибут,obj,"[На, той, встрече, Адель, Нассиф, ,, поняв, ,,..."
3891,,,,,,,,8,,как_CONJ,...,3.0,31005,11,велено_ADV,ADV,,1.0,содержание высказывания,case,"[[, [, Сопровождающий, ,, что, с, усердием, ,,..."
2879,,,Masc,Sing,,,,1,Nom,он_PRON,...,1.0,40649,0,беседовать_VERB,VERB,,-1.0,говорящий,nsubj,"[Беседовать, он, мог, только, на, отвлеченные,..."


In [24]:
N_verify = 2
for i in np.random.choice(len(pd_data), size=N_verify):
    print("-"*60)
    obj = pd_data.iloc[i]
    print(f"Argument position: {obj.arg_address}")
    print(f"Argument lemma: {obj.arg_lemma}")
    print("--")
    print(f"Predicat position: {obj.prd_address}")
    print(f"Predicat lemma: {obj.pred_lemma}")
    print("--")
    print(f"Distance {int(obj.dist)}")
    print("--")
    print(f"Sentence tokens: {list(enumerate(obj.tokens))}")

------------------------------------------------------------
Argument position: 9
Argument lemma: мало_NUM
--
Predicat position: 7
Predicat lemma: становиться_VERB
--
Distance 2
--
Sentence tokens: [(0, 'Сначала'), (1, 'он'), (2, 'оставил'), (3, 'пятерых'), (4, ','), (5, 'потом'), (6, 'их'), (7, 'становилось'), (8, 'всё'), (9, 'меньше'), (10, '.')]
------------------------------------------------------------
Argument position: 4
Argument lemma: ты_PRON
--
Predicat position: 6
Predicat lemma: петь_VERB
--
Distance 2
--
Sentence tokens: [(0, '--'), (1, 'Ну'), (2, 'не'), (3, 'надо'), (4, 'тебе'), (5, 'сейчас'), (6, 'петь'), (7, '.')]


In [25]:
del ling_data_cache
del examples

# Убрать все фичи предикатного слова для модели Unknown Predicates

# Preprocessing

In [26]:
y_stat = pd_data.loc[:, 'role'].value_counts()
drop_ys = y_stat[y_stat < 180].index
clear_data = pd_data.drop(pd_data[pd_data.loc[:, 'role'].isin(drop_ys)].index)

In [27]:
repl_roles = {
    'агенс - субъект восприятия' : 'субъект восприятия',
    'агенс - субъект ментального состояния' : 'субъект ментального состояния',
    'результат / цель' : 'результат',
    'место - пациенс' : 'место',
    'говорящий - субъект психологического состояния' : 'субъект психологического состояния'
}


def normalize_single_region(data, rep, val):
    data.loc[:, 'role'] = data.loc[:, 'role'].str.replace(rep, val)


for rep, val in repl_roles.items():
    normalize_single_region(clear_data, rep, val)
    
number_of_roles = len(clear_data.loc[:, 'role'].value_counts().index)
print('Number of roles: ', number_of_roles)
clear_data.loc[:, 'role'].value_counts()

Number of roles:  44


агенс                                 6147
пациенс                               5362
тема                                  3656
субъект психологического состояния    3250
субъект перемещения                   3011
причина                               2502
говорящий                             2365
место                                 2185
содержание действия                   1874
содержание мысли                      1817
содержание высказывания               1792
конечная точка                        1772
результат                             1452
пациенс перемещения                   1356
стимул                                1271
субъект ментального состояния         1223
адресат                                941
субъект восприятия                     901
контрагент                             831
эффектор                               739
субъект социального отношения          598
начальная точка                        588
предмет высказывания                   548
способ     

In [28]:
clear_data.role.unique()

array(['содержание высказывания', 'говорящий',
       'субъект социального отношения',
       'субъект психологического состояния', 'содержание действия',
       'агенс', 'тема', 'конечная точка', 'сфера', 'контрагент',
       'субъект перемещения', 'причина', 'субъект поведения',
       'ситуация в фокусе', 'исходный посессор',
       'субъект физиологической реакции', 'адресат', 'пациенс', 'срок',
       'источник звука', 'место', 'признак', 'потенциальная угроза',
       'субъект ментального состояния', 'конечный посессор', 'результат',
       'стимул', 'субъект восприятия', 'эффектор', 'траектория',
       'содержание мысли', 'пациенс перемещения', 'каузатор',
       'предмет высказывания', 'начальная точка', 'способ',
       'пациенс социального отношения', 'статус', 'предмет мысли', 'цель',
       'потенциальный пациенс', 'контрагент социального отношения',
       'эталон', 'признак действия'], dtype=object)

In [29]:
y_orig = clear_data.loc[:, 'role']
X_orig = clear_data.drop('role', axis = 1)
X_orig.shape

(52751, 20)

In [30]:
from sklearn.preprocessing import LabelBinarizer
import pickle

label_encoder = LabelBinarizer()
y = label_encoder.fit_transform(y_orig)

with open(main_model_path + '/label_encoder.pckl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [31]:
y.shape

(52751, 44)

## Embedding

In [36]:
from gensim.models import KeyedVectors

embeddings_path = '../../data/ruscorpora_upos_skipgram_300_5_2018.vec'
embeddings = KeyedVectors.load_word2vec_format(embeddings_path, binary=False)
print('Embedding size: ', embeddings.vector_size)

Embedding size:  300


In [37]:
import multiprocessing as mp


def make_embeded_form(word):
    if word:
        #return word[1].encode('utf8')
        return u"{}_{}".format(word[1], word[0])
    else:
        return word


class Embedder_map:
    def __init__(self, embeddings, X):
        self.X_ = X
        self.embeddings_ = embeddings

    def __call__(self, i):  
        result = np.zeros(embeddings.vector_size)
        
        ARG_SPECIAL_TAG = None  # ??

        word = self.X_[i]
        if embeddings.vocab.get(word):
            return embeddings[word]

        return result


def embed(X):
    pool = mp.Pool(4)
    result = pool.map(Embedder_map(embeddings, X), list(range(len(X))), 1000)
    pool.close()
#     embedder = Embedder_map(embeddings, X)
#     result =[embedder(i) for i in range(2)]
#     #result = [embedder(i) for i in range(len(X))]
    return np.asarray(result)

In [38]:
X_orig.columns

Index(['Animacy_arg', 'Aspect_arg', 'Gender_arg', 'Number_arg', 'Tense_arg',
       'Valency_arg', 'VerbForm_arg', 'arg_address', 'arg_case', 'arg_lemma',
       'arg_pos', 'dist', 'ex_id', 'prd_address', 'pred_lemma', 'pred_pos',
       'prepos', 'rel_pos', 'syn_link_name', 'tokens'],
      dtype='object')

In [39]:
%%time

arg_embedded = embed(X_orig['arg_lemma'].values)

CPU times: user 18.5 s, sys: 3.4 s, total: 21.9 s
Wall time: 24.1 s


In [40]:
len(arg_embedded)

52751

In [41]:
%%time

pred_embedded = embed(X_orig['pred_lemma'].values)

CPU times: user 18.2 s, sys: 3.74 s, total: 22 s
Wall time: 24.3 s


In [42]:
np.save("../../data/w2v_verbs.npy", pred_embedded)
np.save("../../data/w2v_args.npy", arg_embedded)

In [ ]:
pred_embedded = np.load("../../data/w2v_verbs.npy")
arg_embedded = np.load("../../data/w2v_args.npy")

In [51]:
class Embedder_single_map:
    def __init__(self, embeddings, X):
        self.X_ = X
        self.embeddings_ = embeddings

    def __call__(self, i):
        #word = make_embeded_form(self.X_[i])
        word = self.X_[i]
        if word in self.embeddings_:
            return self.embeddings_[word]
        else:
            return np.zeros((self.embeddings_.vector_size,))

        
def embed_single(embeddings, X):
    pool = mp.Pool(4)
    result = pool.map(Embedder_single_map(embeddings, X), X.index, 1000)
    pool.close()
        
    return np.asarray(result)

In [ ]:
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz", elmo_output_names=['elmo'])

In [77]:
elmo

In [84]:
elmo(['Я не хочу домой .'.split()])

[array([[-1.3390257 , -0.3563082 , -0.5748834 , ...,  0.76325905,
          0.94826674,  0.8298109 ],
        [-1.1582128 , -0.9295433 ,  0.13361324, ...,  0.35102528,
          0.27033824,  0.06720511],
        [-0.04592073, -0.46310368,  0.0504637 , ...,  0.29876482,
         -0.652665  ,  0.79909015],
        [-0.29748228, -0.4363746 ,  0.33843917, ..., -0.637612  ,
         -0.842638  ,  0.07921451],
        [-0.6751006 , -0.24317859, -0.90687317, ..., -0.24283595,
          0.21287943, -0.35696185]], dtype=float32)]

In [85]:
elmo([['Я']])

[array([[-1.33903   , -0.3545407 , -0.57610404, ...,  0.9724996 ,
          0.27039424,  0.83574843]], dtype=float32)]

In [80]:
elmo([['собака']])

[array([[-0.71904266,  0.22552139,  0.2757006 , ..., -0.31051713,
         -0.5760167 ,  0.50546265]], dtype=float32)]

In [86]:
elmo(['Я не хочу , я не хочу , я не хочу домой .'.split()])[0]

array([[-1.337023  , -0.35688448, -0.5789683 , ...,  0.82202846,
         1.2211776 ,  0.8066084 ],
       [-1.1719527 , -0.9044735 ,  0.13157941, ...,  0.7761724 ,
         0.57176316, -0.33491123],
       [-0.06295706, -0.45263326,  0.05356532, ...,  1.321588  ,
        -0.15975067, -0.07389995],
       ...,
       [ 0.10455199, -0.43030205, -0.09874936, ...,  0.29800093,
        -0.6528196 ,  0.7991791 ],
       [-0.22349963, -0.49061188,  0.21470943, ..., -0.63820523,
        -0.84188545,  0.07996868],
       [-0.6350095 , -0.4391315 , -1.1107223 , ..., -0.24298908,
         0.21316466, -0.35761094]], dtype=float32)

In [88]:
elmo(['Я не хочу , я не хочу , я не хочу домой .'.split()])

[array([[-1.3398    , -0.35434005, -0.57634723, ...,  0.82231534,
          1.2208288 ,  0.80663276],
        [-1.1603794 , -0.93078136,  0.13379881, ...,  0.7763979 ,
          0.57154197, -0.3350079 ],
        [-0.04483984, -0.462993  ,  0.05080022, ...,  1.3217599 ,
         -0.15995294, -0.07382467],
        ...,
        [ 0.1079851 , -0.43433216, -0.10255129, ...,  0.2989173 ,
         -0.65271735,  0.7993736 ],
        [-0.21839234, -0.49216184,  0.21128058, ..., -0.6376457 ,
         -0.8423133 ,  0.07939322],
        [-0.62942576, -0.4276948 , -1.1144581 , ..., -0.24306296,
          0.21245189, -0.35704273]], dtype=float32)]

In [100]:
value = 'что_PRON'

In [101]:
value[:value.rfind('_')]

'что'

In [53]:
def elmo_embed(embeddings, tokens, word_idx1, word_idx2):
    embedded = embeddings([tokens])[0]
    return embedded[min(word_idx1, len(tokens)-1)], embedded[min(word_idx2, len(tokens)-1)] 

In [54]:
obj = X_orig.iloc[38]

In [58]:
verb_idx = obj.prd_address
arg_idx = obj.arg_address
tokens = obj.tokens
embed_verb, embed_arg = elmo_embed(elmo, tokens, verb_idx, arg_idx)

In [77]:
%%time
embedded_verbs = []
embedded_args  = []
for i in tqdm(range(len(X_orig))):
    try:
        if i % 100 == 0:
            with open("./log.txt", 'a', encoding='utf-8') as log:
                print(f"Processed {i} examples", file=log)
        obj = X_orig.iloc[i]
        verb_idx = obj.prd_address
        arg_idx = obj.arg_address
        tokens = obj.tokens
        embed_verb, embed_arg = elmo_embed(elmo, tokens, verb_idx, arg_idx)
        embedded_verbs.append(embed_verb)
        embedded_args.append(embed_arg)
    except Exception as e:
        with open("./log.txt", 'a', encoding='utf-8') as log:
            print(f"Error while processing example {i}={X_orig.iloc[i]}: {e}", file=log)


CPU times: user 2h 40min 22s, sys: 16min 19s, total: 2h 56min 41s
Wall time: 1h 21min 44s


In [78]:
e_verbs = np.stack(embedded_verbs)
e_args  = np.stack(embedded_args)

print(e_verbs.shape, e_args.shape)

(52751, 1024) (52751, 1024)


In [79]:
np.save("../../data/elmo_verbs.npy", e_verbs)
np.save("../../data/elmo_args.npy", e_args)

In [80]:
e_verbs[38]

array([-0.9361637 ,  0.46197   , -0.8599031 , ..., -0.46175578,
        0.61335385, -0.33758122], dtype=float32)

In [43]:
e_verbs = np.load("../../data/elmo_verbs.npy")
e_args = np.load("../../data/elmo_args.npy")

## Vectorizing categorial features

In [44]:
X_orig = X_orig.drop(['tokens'], axis=1)

In [45]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder

#morph_feats = ['pos', 'case', 'anim', 'vform', 'zform', 'shform', 'pform', 'vvform', 'nform', 'time']

# all_feats = (['pred_lemma', 'rel_pos'] + 
#              ['arg_' + e for e in morph_feats] + 
#              ['pred_' + e for e in morph_feats])

# all_feats = (['pred_lemma', 'rel_pos', 'arg_prep'] + 
#              ['arg_' + e for e in morph_feats] + 
#              ['pred_' + e for e in morph_feats])

# all_feats = (['pred_lemma', 'rel_pos', 'arg_prep', 'link_name'] + 
#              ['arg_' + e for e in morph_feats] + 
#              ['pred_' + e for e in morph_feats])

#all_feats = ['pred_lemma', 'rel_pos', 'pred_pos', 'arg_case', 'syn_link_name', 'arg_pos', 'prepos', 'dist']

#categ_feats = [e for e in all_feats if X_orig[e].dtype in [str, object]]
#not_categ = [e for e in all_feats if e not in categ_feats]

#pred_lemma_vectorizer.fit_transform(X_orig.loc[:, ['pred_lemma']].to_dict(orient = 'records'))

not_categ_features = {'arg_address', 'ex_id', 'rel_pos', 'arg_lemma'}
categ_feats = [name for name in X_orig.columns if name not in not_categ_features] 
not_categ = ['rel_pos']
print('Category features:\n', categ_feats)
print('Not category features:\n', not_categ)

# le = LabelEncoder()
# X_train[categorical_cols] = X_train[categorical_cols].apply(lambda col: le.fit_transform(col))
# one_hot_feats = vectorizer.fit_transform(X_orig[categ_feats].to_dict(orient='records'))

vectorizer = DictVectorizer(sparse=False)
one_hot_feats = vectorizer.fit_transform(X_orig[categ_feats].to_dict(orient='records'))
print(one_hot_feats.shape)

with open(main_model_path + '/feature_encoder.pckl', 'wb') as f:
    pickle.dump(vectorizer, f)

Category features:
 ['Animacy_arg', 'Aspect_arg', 'Gender_arg', 'Number_arg', 'Tense_arg', 'Valency_arg', 'VerbForm_arg', 'arg_case', 'arg_pos', 'dist', 'prd_address', 'pred_lemma', 'pred_pos', 'prepos', 'syn_link_name']
Not category features:
 ['rel_pos']
(52751, 817)


In [46]:
np.save("../../data/labnpnpels.npy", y)

In [47]:
not_categ_columns = np.concatenate(tuple(X_orig.loc[:, e].as_matrix().reshape(-1, 1) for e in not_categ), axis =1)
plain_features = np.concatenate((one_hot_feats, not_categ_columns), axis = 1)
plain_features.shape

/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(52751, 818)

In [48]:
np.save("../../data/plain_features.npy", plain_features)

In [49]:
del not_categ_columns
del one_hot_feats

# Model construction

In [50]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Convolution1D, Dropout, MaxPooling1D
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import TimeDistributed
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.layers import RepeatVector
from tensorflow.python.keras.layers import Permute
from tensorflow.python.keras.layers import Lambda
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Concatenate
from tensorflow.python.keras.layers import Bidirectional
from tensorflow.python.keras.layers import Masking
from gensim.models import Word2Vec

In [51]:
def construct_plain_model(input_shape):
    print('Plain model.')
    
    plain_model = Sequential()
    plain_model.add(Dense(600, 
                          #input_shape=(plain_features.shape[1],), 
                          input_shape = input_shape,
                          activation = 'relu'))
    plain_model.add(Dropout(0.3))
    
    plain_model.add(Dense(400))
    plain_model.add(BatchNormalization())
    plain_model.add(Activation('relu'))
    plain_model.add(Dropout(0.3))
    
    plain_model.add(Dense(number_of_roles))
    plain_model.add(BatchNormalization())
    plain_model.add(Activation('softmax'))
    
    plain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return plain_model

In [52]:
def construct_plain_model_sparse(categ_size, emb_size, number_of_roles):    
    input_plain = Input(shape=(categ_size,), name = 'input_categorical')
    input_pred_embed = Input(shape=(emb_size,), name = 'pred_embed')
    input_arg_embed = Input(shape=(emb_size,), name = 'arg_embed')
    
    plain = Dense(400)(input_plain)
    plain = BatchNormalization()(plain)
    plain = Activation('relu')(plain)
    
    def embed_submodel(inpt):
        embed = Dense(100)(inpt)
        embed = BatchNormalization()(embed)
        embed = Activation('relu')(embed)
        return embed
    
    embed_pred = embed_submodel(input_pred_embed)
    embed_arg = embed_submodel(input_arg_embed)
    
    final = Concatenate(axis = 1)([embed_pred, embed_arg, plain])
    final = Dropout(0.3)(final)
    final = Dense(400)(final)
    final = BatchNormalization()(final)
    final = Activation('relu')(final)
    final = Dropout(0.3)(final)
    final = Dense(number_of_roles)(final)
    final = BatchNormalization()(final)
    final = Activation('softmax')(final)
    
    model = Model([input_arg_embed, input_pred_embed, input_plain], final)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Experiments

## Experiments with in-domain test

In [ ]:
# Use only for experiments with in-domain test. Do not use for training

### Plain model

In [55]:
model = construct_plain_model((plain_features.shape[1],))
print(model.summary())
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit(plain_features, y, epochs=15, batch_size=300, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

Plain model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 600)               491400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 400)               240400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 400)               1600      
_________________________________________________________________
activation_2 (Activation)    (None, 400)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 44)                17644   

### Sparse

In [69]:
from keras.models import load_model

In [72]:
from tensorflow.python.keras.models import load_model

In [73]:
test = load_model(os.path.join(main_model_path, 'neural_model.h5'))

In [74]:
test.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pred_embed (InputLayer)         (None, 1024)         0                                            
__________________________________________________________________________________________________
arg_embed (InputLayer)          (None, 1024)         0                                            
__________________________________________________________________________________________________
input_categorical (InputLayer)  (None, 818)          0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 100)          102500      pred_embed[0][0]                 
__________________________________________________________________________________________________
dense_8 (D

In [56]:
model = construct_plain_model_sparse(plain_features.shape[1], 1024, y.shape[1])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([e_args, e_verbs, plain_features], y, epochs=16, batch_size=300, 
          validation_split = 0.1, shuffle=True, callbacks = [early_stopping])
model.save(os.path.join(main_model_path, 'neural_model.h5'))

Train on 47475 samples, validate on 5276 samples
Epoch 1/16
47475/47475 [==============================] - 2s 49us/step - loss: 2.4000 - acc: 0.4630 - val_loss: 1.7841 - val_acc: 0.6439
Epoch 2/16
47475/47475 [==============================] - 1s 21us/step - loss: 1.3658 - acc: 0.7244 - val_loss: 1.3433 - val_acc: 0.7437
Epoch 3/16
47475/47475 [==============================] - 1s 27us/step - loss: 1.0149 - acc: 0.7835 - val_loss: 1.0009 - val_acc: 0.7883
Epoch 4/16
47475/47475 [==============================] - 1s 27us/step - loss: 0.8338 - acc: 0.8126 - val_loss: 0.8103 - val_acc: 0.8069
Epoch 5/16
47475/47475 [==============================] - 1s 23us/step - loss: 0.7097 - acc: 0.8348 - val_loss: 0.7319 - val_acc: 0.8146
Epoch 6/16
47475/47475 [==============================] - 1s 22us/step - loss: 0.6244 - acc: 0.8506 - val_loss: 0.6865 - val_acc: 0.8258
Epoch 7/16
47475/47475 [==============================] - 1s 30us/step - loss: 0.5628 - acc: 0.8605 - val_loss: 0.6413 - val_acc:

In [57]:
from tensorflow.python.keras.backend import clear_session
clear_session()

### Context (NA)

In [ ]:
%%time

model = construct_graph_lstm_model((plain_features.shape[1],))
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([arg_context_embedded, pred_context_embedded, embedded_args, embedded_verbs, plain_features], y, 
          epochs=15, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

## Experiements with out-of-domain test

In [ ]:
# Use only for out of domain experiments. Do not use for training

In [59]:
def evaluate_out_of_domain(model, X_train, y_train, X_test, y_test):
    final_res = list()
    N_ITERATIONS = 5
    for i in xrange(N_ITERATIONS):
        print('Eval iter:', i + 1, '/', N_ITERATIONS)
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, 
                                       patience=2, verbose=0, mode='auto')
        model.fit(X_train, y_train, nb_epoch=15, 
                  batch_size=64, validation_split = 0.1, 
                  shuffle=True, callbacks = [early_stopping],
                 verbose = 0)

        ev_res = evaluate_model(model, X_test, y_test)
        print()
        print(pd.DataFrame([ev_res], columns = ['keras_accur', 'keras_loss', 'f1_micro', 'f1_macro', 'accur']))
        final_res.append(ev_res)
    
    return np.array(final_res)

In [60]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def evaluate_model(model, X_test, y_test):
    keras_eval = model.evaluate(X_test, y_test)
    pred = model.predict(X_test).argmax(axis = 1)
    f1_micro = f1_score(pred, y_test.argmax(axis = 1), average = 'micro')
    f1_macro = f1_score(pred, y_test.argmax(axis = 1), average = 'macro')
    accur = accuracy_score(pred, y_test.argmax(axis = 1))
    
    return np.array(list(keras_eval) + [f1_micro, f1_macro, accur])

### Simple

In [ ]:
model = construct_plain_model((ind_plain_features.shape[1],))
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit(ind_plain_features, ind_y, nb_epoch=15, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

#model.evaluate(ood_plain_features, ood_y)
ev_res = evaluate_model(model, [ood_plain_features], ood_y)
print()
print(pd.DataFrame([ev_res], columns = ['keras_accur', 'keras_loss', 'f1_micro', 'f1_macro', 'accur']))

In [ ]:
model = construct_plain_model((ind_plain_features.shape[1],))
model_eval = evaluate_out_of_domain(model, ind_plain_features, ind_y, ood_plain_features, ood_y)
print(model_eval)
describe_cv_result(model_eval)

### Complex

In [ ]:
model = construct_plain_model_sparse((ind_plain_features.shape[1],))
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([ind_arg_embed, ind_pred_embed, ind_plain_features], ind_y, nb_epoch=20, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])
#model.evaluate([ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)

ev_res = evaluate_model(model, [ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)
print()
print(pd.DataFrame([ev_res], columns = ['keras_accur', 'keras_loss', 'f1_micro', 'f1_macro', 'accur']))

In [ ]:
model = construct_plain_model_sparse((ind_plain_features.shape[1],))
model.summary()
model_eval = evaluate_out_of_domain(model, 
                                    [ind_arg_embed, ind_pred_embed, ind_plain_features], ind_y, 
                                    [ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)
print(model_eval)
describe_cv_result(model_eval)

### Context

In [ ]:
model = construct_graph_lstm_model((ind_plain_features.shape[1],))
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([
           #ind_arg_context, 
        #ind_pred_context,   
        ind_arg_embed, 
        ind_pred_embed, 
        ind_plain_features,
        ind_pred_context], 
#model.fit([ind_arg_context, ind_pred_context, ind_arg_embed, ind_pred_embed, ind_plain_features], 
           ind_y, nb_epoch=6, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

#model.evaluate([ood_arg_context, ood_pred_context, ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)
model.evaluate([
    #    ood_arg_context, 
    #    ood_pred_context,
        ood_arg_embed, 
        ood_pred_embed,
        ood_plain_features,
        ood_pred_context
    ], ood_y)

In [ ]:
model = construct_graph_lstm_model((ind_plain_features.shape[1],))
model.summary()
model_eval = evaluate_out_of_domain(model, 
                                    [ind_arg_embed, ind_pred_embed, ind_plain_features, ind_pred_context], ind_y, 
                                    [ood_arg_embed, ood_pred_embed, ood_plain_features, ood_pred_context], ood_y)
print(model_eval)
describe_cv_result(model_eval)

# Evaluation

In [ ]:
# Use only for model comparision. Do not use for training.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score


def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, *args, **kwargs):
    model.fit(X_train, y_train, *args, **kwargs)
    
    keras_eval = model.evaluate(X_test, y_test)
    
    pred = model.predict(X_test).argmax(axis = 1)
    f1_micro = f1_score(pred, y_test.argmax(axis = 1), average = 'micro')
    f1_macro = f1_score(pred, y_test.argmax(axis = 1), average = 'macro')
    accur = accuracy_score(pred, y_test.argmax(axis = 1))
    
    return list(keras_eval) + [f1_micro, f1_macro, accur]
    

def custom_cross_val(cr_f, X, y, cv, *args, **kwargs):
    cr_f().summary()
    eval_res = list()
    for i, (train, test) in enumerate(cv.split(y)):
        model = cr_f()
        print('Running Fold', i+1, '/', cv.n_splits)
        eval1 = train_and_evaluate_model(model, 
                                         [X[j][train] for j in range(len(X))], y[train], 
                                         [X[j][test] for j in range(len(X))], y[test], 
                                         *args, **kwargs)
        
        print()
        print('Fold result: ', eval1)
        eval_res.append(eval1)
    
    return np.array(eval_res)


def describe_cv_result(cv_res):
    print(cv_res)
    mean_cv_res = cv_res.mean(axis = 0)
    std_cv_res = cv_res.std(axis = 0)
    print('Mean')
    print(pd.DataFrame([mean_cv_res], columns = ['loss', 'keras_accur', 'micro_f1', 'macro_f1', 'accur']))
    print('Std')
    print(pd.DataFrame([std_cv_res], columns = ['loss', 'keras_accur', 'micro_f1', 'macro_f1', 'accur']))
    
    
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=42)

## Simple

In [ ]:
curr_features = np.concatenate((plain_features, e_verbs, e_args), axis = 1)
cv_res = custom_cross_val(lambda : construct_plain_model((curr_features.shape[1],)), 
                          [curr_features], 
                          y, cv = cv, epochs=13, batch_size=64,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

In [ ]:
cv_res = custom_cross_val(lambda : construct_plain_model((plain_features.shape[1],)), 
                          [plain_features], 
                          y, cv = cv, epochs=13, batch_size=64,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

In [ ]:
single_chunk = np.concatenate((e_args, e_verbs, plain_features), axis = 1)
cv_res = custom_cross_val(lambda : construct_plain_model((single_chunk.shape[1],)), 
                          [single_chunk], 
                          y, cv = cv, epochs=13, batch_size=64,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

## Sparse

In [ ]:
cv_res = custom_cross_val(lambda : construct_plain_model_sparse(plain_features.shape[1], 
                                                                embeddings.vector_size, 
                                                                y.shape[1]), 
                          [embedded_args, embedded_verbs, plain_features], y, 
                          cv = cv, epochs=13, batch_size=300,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

## Context

In [ ]:
cv_res = custom_cross_val(lambda : construct_graph_lstm_model((plain_features.shape[1],)), 
                          [arg_context_embedded, 
                           pred_context_embedded, 
                           embedded_args, 
                           embedded_verbs,
                           plain_features], y, 
                          cv = cv, epochs=6, batch_size=64, validation_split = 0., 
                          shuffle=True)

describe_cv_result(cv_res)

# Training and predicting

In [ ]:
# Training model and saving

In [ ]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(X_orig.ex_id.unique(), test_size=0.2, random_state=42)
train_ids = set(train_ids.tolist())
test_ids = set(test_ids.tolist())

In [ ]:
train_selector_pd = X_orig.ex_id.isin(train_ids)
test_selector_pd = X_orig.ex_id.isin(test_ids)
train_selector = train_selector_pd.values
test_selector = test_selector_pd.values

In [ ]:
train_data = {k : data[k] for k in train_ids}
test_data = {k : data[k] for k in test_ids}

with open(os.path.join(main_model_path, 'train_data.json'), 'w') as f:
    json.dump(train_data, f)

with open(os.path.join(main_model_path, 'test_data.json'), 'w') as f:
    json.dump(test_data, f)

In [ ]:
def select_from_nparray_list(nparray_list, selector):
    return [e[selector] for e in nparray_list]

model = construct_plain_model_sparse(plain_features.shape[1], embeddings.vector_size, y.shape[1])
model.fit(select_from_nparray_list([embedded_args, embedded_verbs, plain_features], train_selector),
          select_from_nparray_list([y], train_selector), 
          epochs=10, batch_size=64, validation_split = 0.1, shuffle=True)

model.evaluate(select_from_nparray_list([embedded_args, embedded_verbs, plain_features], test_selector), 
               select_from_nparray_list([y], test_selector))
model.save(os.path.join(main_model_path, 'neural_model.h5'))

In [ ]:
# Hold-out evaluation.

evaluate_model(model,
               select_from_nparray_list([embedded_args, embedded_verbs, plain_features], test_selector), 
               select_from_nparray_list([y], test_selector)[0])

In [ ]:
pred = model.predict(select_from_nparray_list([embedded_args, embedded_verbs, plain_features], test_selector))

test_examples_to_store = X_orig.loc[test_selector_pd[test_selector_pd].index, :].loc[:, ['arg_address', 'ex_id']]
test_data = {k : data[k] for k in test_ids}


for index, (pd_index, row) in enumerate(test_examples_to_store.iterrows()):
    ex = test_data[row['ex_id']]
    arg_addr = row['arg_address']
    sent = ex[arg_addr[0]]
    token = sent[arg_addr[1]]
    cl = pred[index]
    predicted_role = label_encoder.inverse_transform(np.array([cl]))[0]
    actual_role = label_encoder.inverse_transform(np.array([select_from_nparray_list([y], test_selector)[0][index]]))[0]
    
    token['rolepred1'] = actual_role
    token['rolepred2'] = predicted_role

In [ ]:
with open('./test_data_ann_1.json', 'w') as f:
    json.dump(test_data, f)

# Brat convertion

In [ ]:
# Converts results to brat annotation for inspecting.
# Needs framebank_preprocessing from http://nlp.isa.ru/framebank_parser/

In [ ]:
!python2.7 ./framebank_preprocessing/convert_corpus_to_brat.py --inputFile=./test_data_ann_1.json --outputDir=./brat_ann2/

In [ ]:
!python2.7 ./framebank_preprocessing/convert_corpus_to_brat.py --inputFile=./test_data_ann_1.json --outputDir=./syntaxnet_1/ --converter=syn

In [ ]:
!export PYTHONPATH=../ && python2.7 ./convert_corpus_to_brat.py --inputFile=./test_data_ann_1.json --outputDir=./brat_ann2/